PAN CARD VALIDATION

1) DATA CLEANING AND PROCESSING:
   Identify and handle missing data.PAN numbers may have missing values.These missing values need to be handled appropriately either by removing rowa or inputing values
   Check for duplicates and remove them
   PAN numbers may have extra spaces before or after the actual number.Remove any such spaces
   Ensure that the PAN have all uppercase letters
2) PAN FORMAT VALIDATION
   Format: exactly 10 characters: 5 letters, 4 digits, 1 letter → regex ^[A-Z]{5}[0-9]{4}[A-Z]$ (use uppercase; allow auto-upcase).
   Should not have adjacent alphabets or numbers
3) CATEGORIZATION AS VALID AND IVALID PAN
4) TASKS
   Validate the PAN numbers based on format mentioned above.
   Create two seperate catogories:
   *) Valid PAN
   *) Invalid PAN
   Create a summary report that provides:
   1) Total records processed
   2) Total valid PAN's
   3) Total invalid PAN's
   4) Total missing or incomplete PAN's 

In [7]:
!pip install openpyxl



   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [8]:
import pandas as pd

In [110]:
df = pd.read_excel('PAN Number Validation Dataset.xlsx')

In [112]:
df.head(12)
total_records=len(df)
print(total_records)

10000


In [113]:
first_col = df.columns[0]
df[first_col].dtype




dtype('O')

Meaning of dtype('O')

dtype('O') stands for Object type in pandas.

✔ Object dtype usually means:

Text / strings

Mixed data types in the same column

PAN numbers, names, IDs, etc.

So pandas is telling you the first column contains string-like values, not numbers.

In [114]:
df["Pan_Numbers"]=df['Pan_Numbers'].astype('string').str.strip().str.upper()

This will remove extra spaces

In [115]:
df.head(10)

,Pan_Numbers
0,VGLOD3180G
1,PHOXD7232L
2,MGEPH6532A
3,JJCHK4574O
4,XTQIJ2330L
5,HTJYM3835H
6,YQTAP6661X
7,HVOFE5635Y
8,HYUIJ7902R
9,IDSMT3429E


In [116]:
print(df[df['Pan_Numbers'].isna()])

     Pan_Numbers
5022        <NA>
5027        <NA>
5033        <NA>
5043        <NA>
5057        <NA>
...          ...
9961        <NA>
9972        <NA>
9986        <NA>
9987        <NA>
9997        <NA>

[965 rows x 1 columns]


✔ Step 1: df['Pan_Numbers'].isna()

This checks each value in the Pan_Numbers column.

Returns True if the value is NaN (empty/missing).

Returns False if the value is present.

| Pan_Numbers | isna() |
| ----------- | ------ |
| ABCDE1234F  | False  |
| NaN         | True   |
| QWERT5678Y  | False  |


✔ Step 2: df[...]

You give pandas a list of True/False values.

Wherever the condition is True, pandas selects that row.

Wherever it is False, pandas ignores it.

So you get only the rows where Pan_Numbers is missing.




In [117]:
df['Pan_Numbers']=df['Pan_Numbers'].replace('',pd.NA)


Replace empty as NA and drop them using dropna()

In [118]:
df=df.dropna()

In [119]:
print(df[df['Pan_Numbers'].isna()])

Empty DataFrame
Columns: [Pan_Numbers]
Index: []


In [120]:
print('unique pan cards records=',df["Pan_Numbers"].nunique())

unique pan cards records= 9025


In [121]:
print(len(df))

9033


In [122]:
df = df.drop_duplicates(subset='Pan_Numbers',keep='first')

In [123]:
print(len(df))

9025


Keep the first occurrence and delete the rest (default)
and remove the rows that have duplicates of column specified

In [124]:
def has_adjacent_repitition(pan):
#    for i in range(len(pan)-1):
#        if pan[i]==pan[i+1]:
#            return False
#    return True
    return any(pan[i]==pan[i+1] for i in range(len(pan)-1))

    
print(has_adjacent_repitition("APFLC1223Y"))
        
        
        

True


In [125]:
def is_sequential(pan):
#    for i in range(len(pan)-1):
#        if ord(pan[i+1])-ord(pan[i])!=1:
#            return False
#    return True
    return all(ord(pan[i+1])-ord(pan[i])==1 for i in range(len(pan)-1))
print(is_sequential("ABCDE"))
print(is_sequential("MNOPQ"))
print(is_sequential("ABCTFK"))


True
True
False


In [126]:
import re

In [131]:
def is_valid_pan(pan):
    if len(pan)!=10:
        return False
#    if re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$',pan): 
#        return True
    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$',pan): 
        return False
    if has_adjacent_repitition(pan):
        return False
    if is_sequential(pan):
        return False
    return True    
#print(is_valid_pan("ADFHI1782I")) 
#print(is_valid_pan("ADFHI782I")) 
#print(is_valid_pan("A5FHI1782I")) 

df["status"]=df["Pan_Numbers"].apply(lambda x: "valid" if is_valid_pan(x) else "Invalid")
print(df.head(10))
valid_count=(df["status"]=='valid').sum()
invalid_count=(df["status"]=='Invalid').sum()
missing_count=total_records-(valid_count + invalid_count)
print("total records is =",total_records)
print("valid count is =",valid_count)
print("invalid count is =",invalid_count)
print("missing_count is =", missing_count)

df_summary=pd.DataFrame({"TOTAL PROCESSED RECORDS":[total_records],
                         "TOTAL VALID COUNT":[valid_count],
                         "TOTAL INVALID COUNT":[invalid_count],
                         "TOTAL MISSING COUNT":[missing_count]})

print(df_summary.head())
with pd.ExcelWriter("PAN VALIDATION RESULT.xlsx") as writer:
    df.to_excel(writer,sheet_name="PAN_VALIDATION",index=False)
    df_summary.to_excel(writer,sheet_name="SUMMARY",index=False)

  Pan_Numbers   status
0  VGLOD3180G    valid
1  PHOXD7232L    valid
2  MGEPH6532A    valid
3  JJCHK4574O  Invalid
4  XTQIJ2330L  Invalid
5  HTJYM3835H    valid
6  YQTAP6661X  Invalid
7  HVOFE5635Y    valid
8  HYUIJ7902R    valid
9  IDSMT3429E    valid
total records is = 10000
valid count is = 3193
invalid count is = 5832
missing_count is = 975
   TOTAL PROCESSED RECORDS  TOTAL VALID COUNT  TOTAL INVALID COUNT  \
0                    10000               3193                 5832   

   TOTAL MISSING COUNT  
0                  975  
